### Imports

In [2]:
from definitions import *

### View knwoledge test html

In [3]:
url = "https://obrazovaka.ru/test/doktor-zhivago-s-otvetami-online.html"
r = requests.get(url)
soup = BeautifulSoup(r.content, "html.parser")
dict_soup = convert(soup)

In [4]:
with open("dict_soup_obrazovaka.json", "w") as f:
    json.dump(dict_soup, f, indent=2, ensure_ascii=False)

In [15]:
test = []
for quiz_part in dict_soup["html"][0]["body"][0]["div"][0]["div"][0]["div"][0]["div"][0]["article"][0]["div"][5]["div"][1]["ol"][0]["li"]:
    question = quiz_part["h3"][0]["#text"]
    answers_html = quiz_part["ul"][0]["li"]
    answers = []
    for answer_html in answers_html:
        answers.append({
            "answer": answer_html["div"][0]["#text"],
            "is_correct": answer_html["div"][0]["input"][0].get("@data-correct", 0),
        })
    test.append({
        "question": question,
        "answers": answers,
    })

In [17]:
with open("some_test.json", "w") as f:
    json.dump(test, f, indent=2, ensure_ascii=False)

In [12]:
q = [0]["h3"][0]["#text"]
q

'В каких годах был написан роман «Доктор Живаго»?'

In [8]:
answers_html = dict_soup["html"][0]["body"][0]["div"][0]["div"][0]["div"][0]["div"][0]["article"][0]["div"][5]["div"][1]["ol"][0]["li"][0]["ul"][0]["li"]
answers = []
for answer_html in answers_html:
    answers.append(answer_html["div"][0]["#text"])
answers

['1945–1955 гг.', '1946–1956 гг.', '1940–1955 гг.', '1949–1955 гг.']

### Create knowledge tests

In [36]:
def get_knowledge_test(url):
    print("Sleeping before performing request...", end="")
    time.sleep(1)
    clear_output()
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")
    dict_soup = convert(soup)
    test = []
    try:
        quiz = dict_soup["html"][0]["body"][0]["div"][0]["div"][0]["div"][0]["div"][0]["article"][0]["div"][5]["div"][1]["ol"][0]["li"]
    except Exception:
        quiz = dict_soup["html"][0]["body"][0]["div"][0]["div"][0]["div"][0]["div"][0]["article"][0]["div"][6]["div"][1]["ol"][0]["li"]
    for quiz_part in quiz:
        question = quiz_part["h3"][0]["#text"]
        answers_html = quiz_part["ul"][0]["li"]
        answers = []
        for answer_html in answers_html:
            answers.append({
                "answer": answer_html["div"][0]["#text"],
                "is_correct": int(answer_html["div"][0]["input"][0].get("@data-correct", 0)),
            })
        test.append({
            "question": question,
            "answers": answers,
        })
    return test

In [37]:
with open("obrazovaka_quiz_urls.txt", "r") as f:
    obrazovaka_quiz_urls = f.readlines()

In [38]:
obrazovaka_parsed_quizes = []
for url in obrazovaka_quiz_urls:
    obrazovaka_parsed_quizes.append(get_knowledge_test(url))

In [39]:
for i, key in enumerate(BOOKS):
    dir_path = Path("../../metrics/knowledge_tests/")
    dir_path.mkdir(exist_ok=True, parents=True)
    with open(f"{dir_path}/{key}.json", "w") as f:
        json.dump(obrazovaka_parsed_quizes[i], f, indent=2, ensure_ascii=False)